In [ ]:
import torch
import torch.nn as nn
import numpy as np
import h5py
import os

encoding = 'NAVR'

class DB10s(torch.utils.data.Dataset):
    def __init__(self):
        self.h5_list = []
        root='./noise-ecg-classification/data_10s'
        for dir in os.listdir(root):
            if len(dir) != 3:
                dir = os.path.join(root, dir)
                for file in os.listdir(dir):
                  path=os.path.join(dir, file)
                  with h5py.File(path, 'r') as f:
                    if len(f['MLII'][:])==3600:
                      noNoisePath=os.path.join(dir[:-3], file)
                      self.h5_list.append((path, noNoisePath))
        
    def __getitem__(self, index):
        path, noNoisePath = self.h5_list[index]
        with h5py.File(path, 'r') as f, h5py.File(noNoisePath, 'r') as f2:
            MLII = torch.tensor(f['MLII'][:])
            V1 = torch.tensor(f['V1'][:])
            return {
                'MLII': torch.tensor(f['MLII'][:]).float(),
                'V1': torch.tensor(f['V1'][:]).float(),
                'target MLII': torch.tensor(f2['MLII'][:]).float(),
                'target V1': torch.tensor(f2['V1'][:]).float(),
            }
    def __len__(self):
        return len(self.h5_list)
dataset = DB10s()